# Global Optimization

In [1]:
import numpy as np

# Per View Rewards

### Accuracy Reward

In [2]:
def get_accuracy_reward(curr_accuracy, min_accuracy, sigma_right=4, sigma_left=2):
    diff = curr_accuracy - min_accuracy
    if 0<=diff<=1e-2:
        return (np.exp(- (diff**2) / (10 * sigma_left**2)) * 100)
    else:
        return 1*(np.exp(- (abs(diff)**1.5) / (2 * sigma_right**2)) * 100)

### Inference Time Reward

In [3]:
def get_comp_time_reward(current_comp_time, sigma=0.8):
    return np.exp(- (current_comp_time**2) / (2 * sigma**2))*50

### Model Size Reward

In [4]:
def get_model_size_reward(current_model_size, max_model_size, sigma_left=2):
    diff = current_model_size - max_model_size
    if current_model_size > max_model_size:
        return np.exp(- ((diff)**2) / (10 * sigma_left**2))*99
    if current_model_size == max_model_size:
        return 99
    else:
        return (99+(current_model_size/max_model_size))

### Optimal Model Reward

In [5]:
def more_acc_less_size(accuracy, min_accuracy, size, max_model_size):
    if accuracy >= min_accuracy and size <= max_model_size:
        return ((accuracy-min_accuracy)*2) + (max_model_size-size)/2
    return 0

### Per View Final Reward

In [6]:
def get_final_reward(
    min_accuracy, max_model_size, x=1, y=1, z=1,
    curr_accuracy=0, curr_comp_time=0, curr_model_size=0):
    
    acc_reward = get_accuracy_reward(curr_accuracy, min_accuracy)
    comp_time_reward = get_comp_time_reward(curr_comp_time)
    model_size_reward = get_model_size_reward(curr_model_size, max_model_size)
    optimal_model_reward = more_acc_less_size(
        curr_accuracy, min_accuracy, curr_model_size, max_model_size)

    x, y, z = x/(sum([x, y, z]) or 1), y/(sum([x, y, z]) or 1), z/(sum([x, y, z]) or 1)

    return (
        (x * acc_reward)
        + (y * comp_time_reward)
        + (z * model_size_reward)
        + optimal_model_reward
    )

## Global Reward

### Function to find final accuracy of model

#### Inputs

- $i$: Array of $[i_0 \dots i_v]$
- $acc_{-v}$: Array of accuracy of model when view $v$ is removed $[acc_{-0}\dots acc_{-v}]$
- $acc_{v}$: Array of accuracy of using only view $v$ $[arr_0 \dots arr_v]$

### Output
- $acc_f$: Accuracy of final model with all views

1. Find a value for $p$ that optimizes maximizes per-view reward
2. Use $p$ as a function of $i$